---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [15]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [16]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [17]:
def graph_identification():
        return ['PA','SW_L','SW_L','PA','SW_H']
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [18]:
G = nx.read_gpickle('email_prediction.txt')
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [19]:
def salary_predictions():
    d={'ManagementSalary':list(nx.get_node_attributes(G,'ManagementSalary').values()),
       'Department':list(nx.get_node_attributes(G,'Department').values())}
    df=pd.DataFrame(data=d)
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['closeness'] = pd.Series(nx.closeness_centrality(G))
    df['between'] = pd.Series(nx.betweenness_centrality(G))
    dff=df[df['ManagementSalary'].notnull()]
    deff=df[df['ManagementSalary'].isnull()]
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_auc_score
    X=dff[['clustering','degree','closeness','between']]
    y=dff['ManagementSalary']
    X_train,X_test,y_train,y_test=train_test_split(X,y)
    clf=RandomForestClassifier()
    clf.fit(X_train,y_train)
    y_decision=clf.predict(X_test)
    XX_test=deff[['clustering','degree','closeness','between']]
    yy_decision=clf.predict(XX_test)
    return pd.Series( yy_decision.tolist(), index=deff.index)
salary_predictions()

1       0.0
2       1.0
5       1.0
8       0.0
14      0.0
18      0.0
27      0.0
30      1.0
31      0.0
34      0.0
37      0.0
40      0.0
45      0.0
54      0.0
55      0.0
60      0.0
62      1.0
65      1.0
77      0.0
79      0.0
97      0.0
101     0.0
103     0.0
108     0.0
113     0.0
122     0.0
141     0.0
142     1.0
144     0.0
145     1.0
       ... 
913     0.0
914     0.0
915     0.0
918     0.0
923     0.0
926     0.0
931     0.0
934     0.0
939     0.0
944     0.0
945     0.0
947     0.0
950     0.0
951     0.0
953     0.0
959     0.0
962     0.0
963     0.0
968     0.0
969     0.0
974     0.0
984     0.0
987     0.0
989     0.0
991     0.0
992     0.0
994     0.0
996     0.0
1000    0.0
1001    0.0
Length: 252, dtype: float64

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [20]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [21]:
def new_connections_predictions():
    future_connections_tmp = future_connections.copy()
    common_neigh=[(e[0],e[1],len(list(nx.common_neighbors(G,e[0],e[1])))) for e in future_connections_tmp.index]
    jaccard=nx.jaccard_coefficient(G,list(future_connections_tmp.index))
    adamic=nx.adamic_adar_index(G,list(future_connections_tmp.index))
    preferential=nx.preferential_attachment(G,list(future_connections_tmp.index))
    future_connections_tmp['common_neigh']=[x[2] for x in common_neigh]
    future_connections_tmp['jaccard']=[x[2] for x in jaccard]
    future_connections_tmp['adamic']=[x[2] for x in adamic]
    future_connections_tmp['preferential']=[x[2] for x in preferential]    
    dff=future_connections_tmp[future_connections_tmp['Future Connection'].notnull()]
    deff=future_connections_tmp[future_connections_tmp['Future Connection'].isnull()]
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_auc_score
    X=dff[['common_neigh','jaccard','adamic','preferential']]
    y=dff['Future Connection']
    X_train,X_test,y_train,y_test=train_test_split(X,y)
    clf=RandomForestClassifier()
    clf.fit(X_train,y_train)
    y_decision=clf.predict(X_test)
    XX_test=deff[['common_neigh','jaccard','adamic','preferential']]
    yy_decision=clf.predict_proba(XX_test)
    return pd.Series( yy_decision[:,1].tolist(), index=deff.index)       
new_connections_predictions()

(107, 348)    0.000000
(542, 751)    0.014128
(20, 426)     0.300000
(50, 989)     0.013814
(942, 986)    0.009894
(324, 857)    0.013672
(13, 710)     0.000000
(19, 271)     0.300000
(319, 878)    0.011534
(659, 707)    0.014914
(49, 843)     0.011865
(208, 893)    0.004392
(377, 469)    0.000000
(405, 999)    0.000000
(129, 740)    0.000000
(292, 618)    0.000000
(239, 689)    0.004082
(359, 373)    0.244814
(53, 523)     0.100000
(276, 984)    0.005734
(202, 997)    0.012263
(604, 619)    0.000000
(270, 911)    0.011532
(261, 481)    0.000000
(200, 450)    0.600000
(213, 634)    0.020259
(644, 735)    0.100000
(346, 553)    0.000000
(521, 738)    0.011822
(422, 953)    0.000000
                ...   
(672, 848)    0.011532
(28, 127)     1.000000
(202, 661)    0.012875
(54, 195)     1.000000
(295, 864)    0.013679
(814, 936)    0.007439
(839, 874)    0.009894
(139, 843)    0.012746
(461, 544)    0.023499
(68, 487)     0.000000
(622, 932)    0.014993
(504, 936)    0.000000
(479, 528) 